# Autonomous Claims Adjudication - Multi-Agent Sequential Pattern with Strands Agents on AgentCore

### Autonomous Claims Adjudication

Claims adjudication is the process by which insurance companies process, review, validate, and assess claims to determine if they should be paid, adjusted, or denied.

Here is a Generic Example of a Claim Adjudication Workflow for Auto-insurnace

NOTE: THIS IS JUST AN EXAMPLE FOR ILLUSTRATION PURPOSES. 




<p align="center">
    <img src="./images/ClaimsSequentialFlow.png">
</p>

### Agentic Pattern - Sequential

In the Claims Adjudication Flow, there are tasks that have dependencies and   need to be completed before moving to the next task.
We can use a Swarm Sequential pattern where each agent completes its task before passing the result to the next agent in the chain. 


#### Agentic Flow:
An agent that receives an insurance claim (First Notification of Loss), retrieves policy details, validates information against external sources (e.g., repair shop estimates), completes appraisal or damage assessments, and approves payment or flags the claim for human review.

Pattern —> Workflow with Sequential process


<p align="center">
    <img src="./images/ClaimsStrands.drawio.png">
</p>

In [4]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.30.0 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.76 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-mu

In [19]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
!pip freeze | grep boto
!pip freeze | grep agentcore

In [19]:
# Import libraries
import os
import json
import requests
import boto3
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()
region = boto_session.region_name

In [20]:
%store -r

### HELPER FUNCTIONS

In [21]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [22]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

In [23]:
# Load FNOL JSON data
with open('data/FNOL.json', 'r') as f:
    fnol_data = json.load(f)

print("✓ FNOL data loaded successfully")
print(f"\nClaim Number: {fnol_data['fnol']['claimNumber']}")
print(f"Report Date: {fnol_data['fnol']['reportDate']}")
print(f"Policyholder: {fnol_data['policyholder']['firstName']} {fnol_data['policyholder']['lastName']}")
print(f"Vehicle: {fnol_data['vehicle']['insuredVehicle']['year']} {fnol_data['vehicle']['insuredVehicle']['make']} {fnol_data['vehicle']['insuredVehicle']['model']}")

✓ FNOL data loaded successfully

Claim Number: CL-2023-1156789
Report Date: 2023-11-21
Policyholder: John Smith
Vehicle: 2020 Honda Accord


## 1. FNOL PROCESSING

In [24]:
# Create FNOL processing prompt
fnol_prompt = f"""
Please process the following FNOL (First Notice of Loss) data and extract all relevant information.


FNOL Data:
{json.dumps(fnol_data, indent=2)}

"""

print("=" * 80)
print("PROCESSING FNOL DATA")
print("=" * 80)
print("\n FNOL prompt is ready ...\n")

PROCESSING FNOL DATA

 FNOL prompt is ready ...



In [25]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    FNOL_COGNITO_CLIENT_ID,
    FNOL_COGNITO_SECRET
)

In [28]:
fetch_agent_card(FNOL_AGENT_ARN)

{'capabilities': {'streaming': True},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'description': 'Intelligent agent for automated auto insurance claims FNOL processing',
 'name': 'FNOL Data Extraction Specialist',
 'preferredTransport': 'JSONRPC',
 'protocolVersion': '0.3.0',
 'skills': [],
 'url': 'https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A161615149547%3Aruntime%2Ffnol_assistant-MRDFkp2mkG/invocations/',
 'version': '0.0.1'}

### Invoking FNOL Agent and Printing Output

In [27]:
result = await send_sync_message(FNOL_AGENT_ARN, fnol_prompt)
fnol_formatted_output = format_agent_response(result)
print(fnol_formatted_output)

Generated session ID: 28db291a-1cbc-4aee-9f49-375d7a87ab83
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='Intelligent agent for automated auto insurance claims FNOL processing' documentation_url=None icon_url=None name='FNOL Data Extraction Specialist' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[] supports_authenticated_extended_card=None url='https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A161615149547%3Aruntime%2Ffnol_assistant-MRDFkp2mkG/invocations/' version='0.0.1'
# FNOL DATA EXTRACTION RESULTS

## Extracted Structured Claim Information

```json
{
  "fnol": {
    "claimNumber": "CL-2023-1156789",
    "reportDate": "2023-11-21",
    "reportTime": "14:35:00",
    "reportMetho

##  2. APPRAISAL PROCESSING

In [11]:
# Create user prompt with vehicle and damage information
appraisal_input_prompt = f"""
Estimate the repair costs for the following vehicle damage claim. Carefully read and use the {fnol_formatted_output} provided to extract the following required information:

**Claim Information:**
- Claim Number
- Incident Date
- Incident Description

**Insured Vehicle Information:**
- Year
- Make
- Model
- VIN
- Mileage: miles

**Damage Assessment:**
- Damaged Areas
- Damage Description
- Vehicle Drivable

Please provide a detailed repair cost estimate including:
1. Cost breakdown by damaged component
2. Any applicable cost adjustments (luxury brand, vehicle age, etc.)
3. Total estimated repair cost
4. Professional summary and recommendations
"""

print("=" * 80)
print("USER PROMPT CREATED")
print("=" * 80)
# print(appraisal_input_prompt)
print("=" * 80)
print("\n📝 Prompt ready for agent execution")
print("\n" + "=" * 80)

USER PROMPT CREATED

📝 Prompt ready for agent execution



In [12]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    APPRAISAL_COGNITO_CLIENT_ID,
    APPRAISAL_COGNITO_SECRET
)

In [13]:
# fetch_agent_card(APPRAISAL_AGENT_ARN)

### Invoking APPRAISAL agent and printing Output

Use the output of the FNOL Agent as context for input for the Appraisal Agent

In [14]:
appraisal_result = await send_sync_message(APPRAISAL_AGENT_ARN, appraisal_input_prompt)
appraisal_formatted_output = format_agent_response(appraisal_result)
print(appraisal_formatted_output)

Generated session ID: da044208-e7c8-4ea3-aedc-8239a9c2ec8f
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent wit Appraisal tools capabilities' documentation_url=None icon_url=None name='Repair Cost Estimator' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Estimate repair costs for a collision-damaged vehicle based on vehicle type,\ndamaged components, and applicable cost multipliers.\n\nThis tool analyzes vehicle information and damage details to generate a comprehensive\nrepair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),\ncalculates costs for each damaged component, applies multipliers for luxury brands and\nvehicle age, and returns a detailed JSON estimate.\n

## 3. SETTLEMENT ACTIONS
Use the output of the Apprasisal Agent as context for input for the Settlement Agent

In [15]:
bearer_token = reauthenticate_user(
    SETTLEMENT_COGNITO_CLIENT_ID,
    SETTLEMENT_COGNITO_SECRET
)

In [16]:
# fetch_agent_card(SETTLEMENT_AGENT_ARN)

In [17]:

settlement_prompt = f"""Please Settle the claim for the following claim appraisal: {appraisal_formatted_output}"""

### Invoking SETTLEMENT Agent and printing Ouput

In [18]:
settlement_result = await send_sync_message(SETTLEMENT_AGENT_ARN, settlement_prompt)
settlement_formatted_output = format_agent_response(settlement_result)
print(settlement_formatted_output)

Generated session ID: c3ffc1a6-ce94-47b4-b370-580b2b9dae08
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='Intelligent agent for automated auto insurance claims processing and settlement decisions' documentation_url=None icon_url=None name='AutoInsuranceClaimsSettlementAgent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Verifies applicable coverage limits against policy terms.\n\nArgs:\n    claim_number: Unique claim identifier\n    estimated_cost: Total estimated repair cost\n    \nReturns:\n    Coverage verification details with policy limits and applicability', examples=None, id='verify_coverage_limits', input_modes=None, name='verify_coverage_limits', output_modes=None, security=None, tags=[]), AgentSkill(de